#### Notebook to convert GFF to EMBL format for genome submission to ENA

#### Uses [EMBLmyGFF3](https://github.com/NBISweden/EMBLmyGFF3).

#### This notebook was run in an Anaconda enironment for `EMBLmyGFF3` (needed to avoid weird dependency conflicts - likely specific to my Anaconda installation)

In [1]:
%%bash
echo "TODAY'S DATE:"
date
echo "------------"
echo ""
#Display operating system info
lsb_release -a
echo ""
echo "------------"
echo "HOSTNAME: "; hostname 
echo ""
echo "------------"
echo "Computer Specs:"
echo ""
lscpu
echo ""
echo "------------"
echo ""
echo "Memory Specs"
echo ""
free -mh

TODAY'S DATE:
Thu Mar 12 12:31:06 PDT 2020
------------

Distributor ID:	Ubuntu
Description:	Ubuntu 16.04.6 LTS
Release:	16.04
Codename:	xenial

------------
HOSTNAME: 
swoose

------------
Computer Specs:

Architecture:          x86_64
CPU op-mode(s):        32-bit, 64-bit
Byte Order:            Little Endian
CPU(s):                24
On-line CPU(s) list:   0-23
Thread(s) per core:    2
Core(s) per socket:    6
Socket(s):             2
NUMA node(s):          1
Vendor ID:             GenuineIntel
CPU family:            6
Model:                 44
Model name:            Intel(R) Xeon(R) CPU           X5670  @ 2.93GHz
Stepping:              2
CPU MHz:               2925.931
BogoMIPS:              5851.96
Virtualization:        VT-x
L1d cache:             32K
L1i cache:             32K
L2 cache:              256K
L3 cache:              12288K
NUMA node0 CPU(s):     0-23
Flags:                 fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush dts acpi mmx fxsr

No LSB modules are available.


### Set variables
`%env` variables are good for passing to bash cells

In [4]:
# Set workding directory
%env wd=/home/sam/Downloads/20200311_swoose_pgen_genome_ena_submission
wd="/home/sam/Downloads/20200311_swoose_pgen_genome_ena_submission"

# Input output files
%env embl_out=Panopea-generosa-v1.0.a4.embl
%env combined_gff=Panopea-generosa-v1.0.a4.combined.gff
%env fasta=Panopea-generosa-v1.0.fa

# ENA-specific info
%env pid=PRJEB36299
%env locus_tag=PGEN

%env species='Panopea generosa'

env: wd=/home/sam/Downloads/20200311_swoose_pgen_genome_ena_submission
env: embl_out=Panopea-generosa-v1.0.a4.embl
env: combined_gff=Panopea-generosa-v1.0.a4.combined.gff
env: fasta=Panopea-generosa-v1.0.fa
env: pid=PRJEB36299
env: locus_tag=PGEN
env: species='Panopea generosa'


In [5]:
cd {wd}

/home/sam/Downloads/20200311_swoose_pgen_genome_ena_submission


#### All GFF files were downloaded from here: https://osf.io/yem8n/files/

In [6]:
%%bash
ls -ltrh

total 1.1G
-rw-rw-r-- 1 sam sam 914M Mar  3 15:08 Panopea-generosa-v1.0.fa
-rw-rw-r-- 1 sam sam  217 Mar  3 15:09 pgen_v074_ena_submission_manifest.tab
-rw-rw-r-- 1 sam sam  658 Mar  3 21:08 Panopea-generosa-v1.0.fa.fai
-rw-rw-r-- 1 sam sam  53M Mar  4 06:31 Panopea-generosa-v1.0.a4.CDS.gff3
-rw-rw-r-- 1 sam sam  55M Mar  4 06:32 Panopea-generosa-v1.0.a4.exon.gff3
-rw-rw-r-- 1 sam sam 9.1M Mar  4 06:32 Panopea-generosa-v1.0.a4.mRNA.gff3
-rw-rw-r-- 1 sam sam 1.5K Mar  4 06:32 Panopea-generosa-v1.0.a4.rRNA.gff3
-rw-rw-r-- 1 sam sam 2.6M Mar  4 06:32 Panopea-generosa-v1.0.a4.tRNA.gff3
-rw-rw-r-- 1 sam sam 5.4M Mar  4 06:32 Panopea-generosa-v1.0.a4.repeats.DNA.gff3
-rw-rw-r-- 1 sam sam  18M Mar  4 06:33 Panopea-generosa-v1.0.a4.repeats.LINE.gff3
-rw-rw-r-- 1 sam sam 763K Mar  4 06:33 Panopea-generosa-v1.0.a4.repeats.LTR.gff3
-rw-rw-r-- 1 sam sam 143K Mar  4 06:33 Panopea-generosa-v1.0.a4.repeats.RC.gff3
-rw-rw-r-- 1 sam sam 4.6M Mar  4 06:33 Panopea-generosa-v1.0.a4.repeats.Simple_repeat.g

#### Concatenate GFFs to create singular GFF

This step also changes the text "notes=" to "note=". This will allow that section of the GFF to be included in the output EMBL flat file. The info in this section includes information on SwissProt, InterProScan, and Pfam accession numbers.

In [7]:
%%bash
# Copy GFF3 header to new GFF3
awk 'NR < 4' Panopea-generosa-v1.0.a4.CDS.gff3 > ${combined_gff}

echo "GFF line counts:"
echo ""

# Concatenate all GFFs
for gff in *.gff3
do
  # Print file name and line count (excluding header)
  echo "${gff}"
  awk 'NR > 3' ${gff} | wc -l

  # Skip header and substitute "notes=" with "note="
  awk 'NR > 3' ${gff} | sed 's/notes=/note=/g' >> ${combined_gff}
done

echo ""
echo "-------------------------------"
echo ""

echo "${combined_gff} line count:"
awk 'NR > 3' ${combined_gff} | wc -l

GFF line counts:

Panopea-generosa-v1.0.a4.CDS.gff3
236960
Panopea-generosa-v1.0.a4.exon.gff3
236960
Panopea-generosa-v1.0.a4.gene.gff3
34947
Panopea-generosa-v1.0.a4.mRNA.gff3
38326
Panopea-generosa-v1.0.a4.repeats.DNA.gff3
23195
Panopea-generosa-v1.0.a4.repeats.LINE.gff3
75939
Panopea-generosa-v1.0.a4.repeats.LTR.gff3
3255
Panopea-generosa-v1.0.a4.repeats.RC.gff3
603
Panopea-generosa-v1.0.a4.repeats.Simple_repeat.gff3
19865
Panopea-generosa-v1.0.a4.repeats.SINE.gff3
43129
Panopea-generosa-v1.0.a4.rRNA.gff3
8
Panopea-generosa-v1.0.a4.tRNA.gff3
16889

-------------------------------

Panopea-generosa-v1.0.a4.combined.gff line count:
730076


#### Display program options

In [8]:
%%bash
EMBLmyGFF3 -h

usage: EMBLmyGFF3 [-h] [-a] [-c CREATED]
                  [-d {CON,PAT,EST,GSS,HTC,HTG,MGA,WGS,TSA,STS,STD}]
                  [-g ORGANELLE] [-i LOCUS_TAG] [-k KEYWORD [KEYWORD ...]]
                  [-l CLASSIFICATION]
                  [-m {genomic DNA,genomic RNA,mRNA,tRNA,rRNA,other RNA,other DNA,transcribed RNA,viral cRNA,unassigned DNA,unassigned RNA}]
                  [-o OUTPUT] [-p PROJECT_ID] [-q]
                  [-r {1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25}]
                  [-s SPECIES] [-t {linear,circular}] [-v]
                  [-x {PHG,ENV,FUN,HUM,INV,MAM,VRT,MUS,PLN,PRO,ROD,SYN,TGN,UNC,VRL}]
                  [-z] [--ah {One of the parameters above}] [--de DE]
                  [--ra RA [RA ...]] [--rc RC] [--rg RG] [--rl RL] [--rt RT]
                  [--rx RX] [--email EMAIL] [--expose_translations]
                  [--force_unknown_features] [--force_uncomplete_features]
                  [--interleave_genes] [--keep_duplicates]
 

#### Convert combined GFF to EMBL flat file

In [11]:
%%bash
time \
EMBLmyGFF3 ${combined_gff} \
${fasta} \
--topology linear \
--molecule_type 'genomic DNA' \
--transl_table 2 \
--species "${species}" \
--locus_tag ${locus_tag} \
--project_id ${pid} \
--output ${embl_out}

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [12]:
%%bash
head ${embl_out}

ID   XXX; XXX; linear; genomic DNA; XXX; XXX; 89643857 BP.
XX
AC   XXX; 
XX
AC * _Scaffold_01
XX
PR   Project:PRJEB36299;
XX
DT   12-Mar-2020 (Rel. 133, Created)
XX


In [13]:
%%bash
head -n 50 ${embl_out}

ID   XXX; XXX; linear; genomic DNA; XXX; XXX; 89643857 BP.
XX
AC   XXX; 
XX
AC * _Scaffold_01
XX
PR   Project:PRJEB36299;
XX
DT   12-Mar-2020 (Rel. 133, Created)
XX
DE   XXX
XX
KW   .
XX
OS   'panopea generosa'
XX
OC   Life.
XX
RN   [1]
RP   1-89643857
RG   XXX
RT   ;
RL   Submitted (12-MAR-2020) to the INSDC.
XX
FH   Key             Location/Qualifiers
FH
FT   source          1..89643857
FT                   /mol_type="genomic DNA"
FT                   /organism="'panopea generosa'"
FT   gene            2..4719
FT                   /locus_tag="PGEN_LOCUS1"
FT                   /note="sp|Q86IC9|CAMT1_DICDI [BLAST protein vs protein
FT                   (blastp) 2.7.1],PF01596.12 [Pfam 1.6]"
FT                   /note="source:GenSAS_5d9637f372b5d-publish"
FT                   /note="ID:PGEN_.00g000010"
FT                   /standard_name="PGEN_.00g000010"
FT   mRNA            join(2..125,1995..2095,3325..3495,4651..4719)
FT                   /locus_tag="PGEN_LOCUS1"
FT                  

In [14]:
%%bash
ls -ltrh

total 2.6G
-rw-rw-r-- 1 sam sam 914M Mar  3 15:08 Panopea-generosa-v1.0.fa
-rw-rw-r-- 1 sam sam  658 Mar  3 21:08 Panopea-generosa-v1.0.fa.fai
-rw-rw-r-- 1 sam sam  53M Mar  4 06:31 Panopea-generosa-v1.0.a4.CDS.gff3
-rw-rw-r-- 1 sam sam  55M Mar  4 06:32 Panopea-generosa-v1.0.a4.exon.gff3
-rw-rw-r-- 1 sam sam 9.1M Mar  4 06:32 Panopea-generosa-v1.0.a4.mRNA.gff3
-rw-rw-r-- 1 sam sam 1.5K Mar  4 06:32 Panopea-generosa-v1.0.a4.rRNA.gff3
-rw-rw-r-- 1 sam sam 2.6M Mar  4 06:32 Panopea-generosa-v1.0.a4.tRNA.gff3
-rw-rw-r-- 1 sam sam 5.4M Mar  4 06:32 Panopea-generosa-v1.0.a4.repeats.DNA.gff3
-rw-rw-r-- 1 sam sam  18M Mar  4 06:33 Panopea-generosa-v1.0.a4.repeats.LINE.gff3
-rw-rw-r-- 1 sam sam 763K Mar  4 06:33 Panopea-generosa-v1.0.a4.repeats.LTR.gff3
-rw-rw-r-- 1 sam sam 143K Mar  4 06:33 Panopea-generosa-v1.0.a4.repeats.RC.gff3
-rw-rw-r-- 1 sam sam 4.6M Mar  4 06:33 Panopea-generosa-v1.0.a4.repeats.Simple_repeat.gff3
-rw-rw-r-- 1 sam sam  11M Mar  4 06:33 Panopea-generosa-v1.0.a4.repeats.S

#### Clean up

In [15]:
%%bash
rm *.gff *.gff3
ls -ltrh

total 2.3G
-rw-rw-r-- 1 sam sam 914M Mar  3 15:08 Panopea-generosa-v1.0.fa
-rw-rw-r-- 1 sam sam  658 Mar  3 21:08 Panopea-generosa-v1.0.fa.fai
-rw-rw-r-- 1 sam sam  205 Mar 12 13:41 pgen_v074_ena_submission_manifest.tab
-rw-rw-r-- 1 sam sam 1.4G Mar 12 13:47 Panopea-generosa-v1.0.a4.embl


#### Mainfest file required for submission

In [16]:
%%bash
cat pgen_v074_ena_submission_manifest.tab

STUDY   PRJEB36299
SAMPLE   ERS4366161
ASSEMBLYNAME   PGEN-v1.0
ASSEMBLY_TYPE isolate
COVERAGE   86X
PROGRAM   Proximo
PLATFORM   Hi-C,HiSeq4000
MOLECULETYPE   genomic DNA
FLATFILE   Panopea-generosa-v1.0.a4.embl


In [17]:
%%bash
cat pgen_v074_ena_submission_manifest.tab

STUDY   PRJEB36299
SAMPLE   ERS4366161
ASSEMBLYNAME   PGEN-v1.0
ASSEMBLY_TYPE isolate
COVERAGE   86
PROGRAM   Proximo
PLATFORM   Hi-C,HiSeq4000
MOLECULETYPE   genomic DNA
FLATFILE   Panopea-generosa-v1.0.a4.embl


#### ENA requires gzipped or bzipped files

In [19]:
%%bash
time \
gzip Panopea-generosa-v1.0.a4.embl

ls -ltrh

total 1.3G
-rw-rw-r-- 1 sam sam 914M Mar  3 15:08 Panopea-generosa-v1.0.fa
-rw-rw-r-- 1 sam sam  658 Mar  3 21:08 Panopea-generosa-v1.0.fa.fai
-rw-rw-r-- 1 sam sam 339M Mar 12 13:47 Panopea-generosa-v1.0.a4.embl.gz
-rw-rw-r-- 1 sam sam  212 Mar 12 15:53 pgen_v074_ena_submission_manifest.tab
-rw-rw-r-- 1 sam sam  295 Mar 12 15:53 pgen_v074_ena_submission_manifest.tab.report
-rw-rw-r-- 1 sam sam  511 Mar 12 15:53 webin-cli.report



real	2m20.001s
user	2m19.208s
sys	0m0.788s
